In [ ]:
import pandas as pd

from util import Pipeline

p = Pipeline('configs')

In [ ]:
ofm2019 = p.get_table('ofm_estimates_2019')

In [ ]:
p.get_id_col('ofm_estimates_2019')

In [ ]:
for file in p.get_elmer_geo_list():
    if 'id_col' in file:
        print(file['name'].rstrip('s') + '_id', file['id_col'])

In [ ]:
blk = p.get_geodataframe('blocks')

In [ ]:
p.settings['ElmerGeo']

In [ ]:
blk